In [1]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [2]:
if not (os.path.exists('./Data/preprocessed_user_item_rating.csv')):
  raise ValueError('El archivo preprocessed_user_item_rating.csv no fue encontrado en el path')
else:
  print("El archivo ha sido cargado")

El archivo ha sido cargado


In [3]:
ratings=pd.read_csv('./Data/preprocessed_user_item_rating.csv', sep = ',', header=0, names = [ '', 'userid', 'artist-name', 'count', 'max', 'rating' ] )
ratings = ratings.loc[:,['userid', 'artist-name','rating']]
ratings

,userid,artist-name,rating
0,user_001000,Wilco,5.0
1,user_001000,Radiohead,4.9
2,user_001000,Animal Collective,4.7
3,user_001000,Girl Talk,4.6
4,user_001000,Aesop Rock,4.2
...,...,...,...
894494,user_000001,曽我部恵一,0.0
894495,user_000001,畠山美由紀,0.0
894496,user_000001,細野晴臣,0.0
894497,user_000001,青柳拓次,0.0


# Creacion sistema de recomendacion

In [8]:
reader = Reader( rating_scale = ( 0, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'userid', 'artist-name', 'rating' ] ], reader )

In [69]:
trainset, testset=  train_test_split(surprise_dataset, test_size=.2)

### Modelo basado en distancias coseno

In [70]:
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=10, min_k=5, sim_options=sim_options)

### Modelo basado en índice de Jaccard

### Modelo basado en correlación de Pearson

In [41]:
sim_options = {'name': 'pearson_baseline',
               'user_based': True,
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

### Recomendaciones

In [71]:
predictions = algo.fit(trainset).test(testset)

Computing the cosine similarity matrix...


/Users/juansebastianmendezrubiano/.local/share/virtualenvs/SistemasRecomendacion_Taller1-HmHlzIuN/lib/python3.8/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [72]:
#Predicciones para usuario user_001000
user_predictions=list(filter(lambda x: x[0]=='user_001000',predictions))

In [73]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [74]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [75]:
#Se convierte a dataframe
labels = ['artist', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

### Recomendaciones usuario 001000

In [76]:
df_predictions

,artist,estimation
0,John Maus,1.251473
1,Broken Social Scene,1.216894
2,Devendra Banhart,1.087251
3,Elliott Smith,0.900362
4,Rilo Kiley,0.892150
5,The Shins,0.813107
6,Spoon,0.767754
7,The Decemberists,0.759360
8,Shugo Tokumaru,0.705304
9,Animal Collective,0.673654


In [77]:
accuracy.rmse( user_predictions, verbose = True )

RMSE: 1.4194


1.4194457469580022

In [79]:
algo.predict('user_001000','John Maus')

Prediction(uid='user_001000', iid='John Maus', r_ui=None, est=1.2514728871977372, details={'actual_k': 9, 'was_impossible': False})

In [82]:
ratings_100 = ratings[ratings['userid']=='user_001000']
ratings_100[ratings_100['artist-name']=='Spoon']

,userid,artist-name,rating
54,user_001000,Spoon,1.1
